In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install opencv-python

In [ ]:
!pip install pyodbc

In [ ]:
# Clone YOLOv5 repo if not already cloned

import os

if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5
    %cd yolov5
    !pip install -r requirements.txt
    %cd ..
else:
    print("YOLOv5 repo already exists.")
    
if not os.path.exists('yolov5/runs'):
    os.rename('runs', 'yolov5/runs')

In [ ]:
import torch
import cv2
import time
from Alert import alert_notification  # Import alert system

# Load custom trained YOLOv5 model
model = torch.hub.load('yolov5', 'custom', path='yolov5/runs/train/yawn-detection7/weights/best.pt', source='local')
model.conf = 0.5  # Confidence threshold

# Set driver name
driver_name = "Driver_1"  

# Open webcam
cap = cv2.VideoCapture(0)

# Cooldown to avoid spamming multiple alerts for the same yawn
alert_cooldown = 5  
last_alert_time = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    for det in results.xyxy[0]:  # detections per image
        class_id = int(det[5])  # 5th index is class
        confidence = float(det[4])  # 4th index is confidence
        if model.names[class_id] == 'Yawn':
            current_time = time.time()
            if current_time - last_alert_time > alert_cooldown:
                print("Yawn detected! Sending alert...")
                alert_notification(driver_name)
                last_alert_time = current_time

            # Draw box
            x1, y1, x2, y2 = map(int, det[:4])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, 'Yawn', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Show live feed
    cv2.imshow('3DSP Driver Monitoring', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()